In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import multiprocessing
from sentence_transformers import SentenceTransformer

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
headphone_electronics = pd.read_pickle('merged_df.pkl')

In [3]:
headphone_electronics

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,style_json,brand
0,5.0,"02 22, 2015",A38RQFVQ1AKJQQ,4126895493,{'Color:': ' Blue W/Mic'},George Walker,great headphones cord short,"{""Color:"": "" Blue W/Mic""}",HeadGear
1,5.0,"05 8, 2017",A299MRB9O6GWDE,4126895493,{'Color:': ' Blue Zebra W/Mic'},Carolyn B,really like headphone wanted something office ...,"{""Color:"": "" Blue Zebra W/Mic""}",HeadGear
2,1.0,"11 5, 2016",A3ACFC6DQQLIQT,4126895493,{'Color:': ' Blue W/Mic'},MK,wire headphone break less month dont buy,"{""Color:"": "" Blue W/Mic""}",HeadGear
3,3.0,"09 24, 2016",A36BC0YFDBNB5X,4126895493,{'Color:': ' Green'},bigboy,good,"{""Color:"": "" Green""}",HeadGear
4,1.0,"07 17, 2016",A212PQ0HQPNNWM,4126895493,{'Color:': ' Violet Purple'},Kelly Hales,currently return product sound doesnt come spe...,"{""Color:"": "" Violet Purple""}",HeadGear
...,...,...,...,...,...,...,...,...,...
625426,5.0,"04 10, 2017",A23BQ4GZNFBLET,B01HI9GG30,nan,John,love aux cord genius finally use spring trick ...,NaN,Beautiful-tech
625427,3.0,"02 11, 2017",A205H5OQKYCPSH,B01HI9GG30,nan,Apex,get crackle pop occasionally,NaN,Beautiful-tech
625428,5.0,"01 17, 2017",AIW5NLEN5V40Y,B01HI9GG30,nan,ray sooley,great cable need right angle brookstone ampspe...,NaN,Beautiful-tech
625429,5.0,"10 8, 2016",A39UA7Q4FLM0DL,B01HI9GG30,nan,binkisoto799,work perfect purchase husband car hook phone v...,NaN,Beautiful-tech


# Get Embeddings of each reviewText using Word2Vec

In [4]:
from sentence_transformers import SentenceTransformer

# Load pre-trained BERT-based model
model = SentenceTransformer('bert-base-uncased')

No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def get_bert_embeddings(text):
    # Generate embeddings
    embeddings = model.encode(text)
    return embeddings

In [4]:
if not os.path.exists('embeddings'):
    os.makedirs('embeddings')

# Iterate through each row in the dataframe
for index, row in headphone_electronics.iterrows():
    review_text = row['reviewText']  # Assuming 'reviewText' is the column containing the review text
    embeddings = get_bert_embeddings(review_text)
    # Save embeddings to a numpy file
    np.save(f'embeddings/embeddings_{index}.npy', embeddings)

In [5]:
# Create an empty list to store rows
rows = []

# Define the folder containing the embeddings
embedding_folder = 'embeddings/'

# Define the number of rows to iterate over
num_rows = 101555

# Iterate through each row in the dataframe up to the specified number of rows
for index, row in headphone_electronics.head(num_rows).iterrows():
    # Generate the path to the embedding file
    embedding_path = os.path.join(embedding_folder, f'embeddings_{index}.npy')
    
    # Get the rating from the 'overall' column
    rating = row['overall']
    
    # Append the embedding path and rating to the list of rows
    rows.append({'Embedding Path': embedding_path, 'Rating': rating})

# Create a DataFrame from the list of rows
embedding_rating_df = pd.DataFrame(rows)


In [8]:
embedding_rating_df

,Embedding Path,Rating,Rating Class
0,embeddings/embeddings_0.npy,5.0,Good
1,embeddings/embeddings_1.npy,5.0,Good
2,embeddings/embeddings_2.npy,1.0,Bad
3,embeddings/embeddings_3.npy,3.0,Average
4,embeddings/embeddings_4.npy,1.0,Bad
...,...,...,...
101550,embeddings/embeddings_101550.npy,4.0,Good
101551,embeddings/embeddings_101551.npy,5.0,Good
101552,embeddings/embeddings_101552.npy,5.0,Good
101553,embeddings/embeddings_101553.npy,2.0,Bad


In [7]:
#Classify the above ratings
# Create an empty list to store the classified ratings
rating_class = []

# Iterate through each row in the dataframe up to the specified number of rows
for index, row in embedding_rating_df.iterrows():
    # Get the rating from the 'overall' column
    rating = row['Rating']
    
    # Classify the rating
    if rating > 3:
        rating_class.append('Good')
    elif rating == 3:
        rating_class.append('Average')
    else:
        rating_class.append('Bad')

# Add the 'Rating Class' column to the DataFrame
embedding_rating_df['Rating Class'] = rating_class


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Create an empty list to store the embeddings
embeddings = []

# Define the folder containing the embeddings
embedding_folder = 'embeddings/'

# Iterate through each embedding path
for embedding_path in embedding_rating_df['Embedding Path']:
    # Load the embedding from the .npy file
    embedding = np.load(embedding_path)
    # Append the embedding to the list of embeddings
    embeddings.append(embedding)

# Convert the list of embeddings to a NumPy array
X = np.array(embeddings)

# Define the target variable
y = embedding_rating_df['Rating Class']

# Split the data into training and testing sets (75:25 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Optionally, you can print the shapes of the training and testing sets to verify
#print("X_train shape:", X_train.shape)
#print("X_test shape:", X_test.shape)
#print("y_train shape:", y_train.shape)
#print("y_test shape:", y_test.shape)


In [11]:
import pickle
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier

# Define the models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(),
}

# Train, evaluate each model, and save results
for model_name, model in models.items():
    if model_name == 'XGBoost':
        # Encode labels for XGBoost
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)
        # Train the XGBoost model
        model.fit(X_train, y_train_encoded)
    else:
        # Train other models without label encoding
        model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate precision, recall, F1-score, and support
    if model_name == 'XGBoost':
        precision, recall, f1, support = precision_recall_fscore_support(y_test_encoded, y_pred, labels=[0, 1, 2])
        classes = label_encoder.classes_
    else:
        precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, labels=['Good', 'Average', 'Bad'])
        classes = ['Good', 'Average', 'Bad']
        
    # Store the results
    results = {
        'Model': model_name,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'Support': support
    }

    # Print the results
    print(f"Model: {model_name}")
    for cls, metric_values in zip(['Good', 'Average', 'Bad'], [precision, recall, f1, support]):
        print(f"Class: {cls}")
        print(f"  Precision: {metric_values[0]}")
        print(f"  Recall: {metric_values[1]}")
        print(f"  F1-score: {metric_values[2]}")
        if len(metric_values) > 3:
            print(f"  Support: {metric_values[3]}")
        print()

    # Save results to a pickle file
    with open(f'{model_name.lower().replace(" ", "_")}_results.pkl', 'wb') as f:
        pickle.dump(results, f)
        print("Results Saved")


Model: Logistic Regression
Class: Good
  Precision: 0.8309923594271857
  Recall: 0.38441558441558443
  F1-score: 0.6256752462662853

Class: Average
  Precision: 0.9577620755114955
  Recall: 0.05597579425113464
  F1-score: 0.5207617032531077

Class: Bad
  Precision: 0.8898851081551162
  Recall: 0.09772202046880159
  F1-score: 0.568418013856813

Results Saved
Model: Decision Tree
Class: Good
  Precision: 0.8028267411865864
  Recall: 0.15125173852573018
  F1-score: 0.3142125480153649

Class: Average
  Precision: 0.7877557477325459
  Recall: 0.1645234493192133
  F1-score: 0.32451732345940226

Class: Bad
  Precision: 0.7952198445651018
  Recall: 0.1576086956521739
  F1-score: 0.3192818110850898

Results Saved
Model: Random Forest
Class: Good
  Precision: 0.7703503822723742
  Recall: 0.6774193548387096
  F1-score: 0.7638248847926268

Class: Average
  Precision: 0.9935667580679182
  Recall: 0.01588502269288956
  F1-score: 0.17535043639248876

Class: Bad
  Precision: 0.8678350183082697
  Recal

In [ ]:
results.to_pickle('results.pkl')

In [3]:
results_decision = pd.read_pickle('decision_tree_results.pkl')
results_decision

{'Model': 'Decision Tree',
 'Precision': array([0.80255017, 0.14823944, 0.31347804]),
 'Recall': array([0.78659565, 0.15922844, 0.32848453]),
 'F1-score': array([0.79449282, 0.15353756, 0.32080589]),
 'Support': array([18964,  2644,  3781])}

In [4]:
results_logistic = pd.read_pickle('logistic_regression_results.pkl')
results_logistic

{'Model': 'Logistic Regression',
 'Precision': array([0.83099236, 0.38441558, 0.62567525]),
 'Recall': array([0.95776208, 0.05597579, 0.5207617 ]),
 'F1-score': array([0.88988511, 0.09772202, 0.56841801]),
 'Support': array([18964,  2644,  3781])}

In [5]:
results_knn = pd.read_pickle('knn_results.pkl')
results_knn

{'Model': 'KNN',
 'Precision': array([0.80099163, 0.1882662 , 0.51966416]),
 'Recall': array([0.92854883, 0.08131619, 0.31102883]),
 'F1-score': array([0.86006643, 0.11357633, 0.38914626]),
 'Support': array([18964,  2644,  3781])}

In [6]:
results_random = pd.read_pickle('random_forest_results.pkl')
results_random

{'Model': 'Random Forest',
 'Precision': array([0.77000736, 0.68253968, 0.75465116]),
 'Recall': array([0.99340856, 0.01626324, 0.17164771]),
 'F1-score': array([0.86755699, 0.03176949, 0.2796811 ]),
 'Support': array([18964,  2644,  3781])}

In [7]:
results_xgboost = pd.read_pickle('xgboost_results.pkl')
results_xgboost

{'Model': 'XGBoost',
 'Precision': array([0.30327869, 0.63011748, 0.82011344]),
 'Recall': array([0.04198185, 0.46813012, 0.96066231]),
 'F1-score': array([0.07375415, 0.53717754, 0.88484142]),
 'Support': array([ 2644,  3781, 18964])}